In [7]:
# Imports
import sys
import os
import numpy as np
import pandas as pd
import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from numpy import argmax
import seaborn as sns

from imp import reload
import utils
reload(utils)
from utils import*

import matplotlib.pyplot as plt
%matplotlib inline
import csv
import scipy as sp
import getFeatures as gf
import mne

from pathlib import Path

In [8]:
working_dir = Path.cwd()
data_dir = working_dir

In [9]:

def csv_to_mne_raw(csv_file_path):
  #read the csv file into a pandas dataframe
  df = pd.read_csv(csv_file_path)

  #get the starting 10 columns corresponding to the 8 channels and the index data
  df1 = df.iloc[:,0:10]

  #also keep the timestamp, Cognitive Workload and Anxiety column
  df1['Timestamp'] = pd.to_datetime(df[' Timestamp (Formatted)'])
  df1['Cognitive Workload'] = df['Cognitive_Wokload']
  df1['Anxiety'] = df['Anxiety']

  #reindex the columns
  df1 = df1.reindex(columns=[' EXG Channel 0', ' EXG Channel 1', ' EXG Channel 2',
                            ' EXG Channel 3', ' EXG Channel 4', ' EXG Channel 5', 
                            ' EXG Channel 6', ' EXG Channel 7', 'Changed Sample Index',
                            'Sample Index','Cognitive Workload','Anxiety','Timestamp'])


  #drop the sample index column
  df1 = df1.drop(['Sample Index'], axis=1)

  #rename the channels
  df1 = df1.rename(columns={' EXG Channel 0': 'eeg1', ' EXG Channel 1': 'eeg2',
                            ' EXG Channel 2': 'eeg3', ' EXG Channel 3': 'eeg4',
                              ' EXG Channel 4': 'eeg5', ' EXG Channel 5': 'eeg6',
                              ' EXG Channel 6': 'eeg7', ' EXG Channel 7': 'eeg8',
                              'Changed Sample Index':'Index'})


  #convert dataframe to numpy array
  data = df1[['eeg1','eeg2','eeg3','eeg4','eeg5','eeg6','eeg7','eeg8']].to_numpy()
  # data = df1.to_numpy()

  #convert the data to float32
  data = data.astype('float32')

  # Scaling Data with the scale factor S = (4500000)/4/(2**23-1)
  data = data*4500000/4/(2**23-1)

  #get transpose of the data
  data = data.T

  # Create some metadata
  n_channels = 8
  sampling_freq = 250  # in Hertz
  ch_types = ['eeg'] * n_channels
  ch_names = ['eeg1', 'eeg2', 'eeg3', 'eeg4', 'eeg5', 'eeg6', 'eeg7', 'eeg8']
  info = mne.create_info( ch_names=ch_names, sfreq=sampling_freq,ch_types=ch_types)
  # print(info)
  raw = mne.io.RawArray(data, info)
  return raw,df1.iloc[0]['Cognitive Workload'] 

  


In [4]:
# Create some metadata
n_channels = 8
sampling_freq = 250  # in Hertz
ch_types = ['eeg'] * n_channels
ch_names = ['eeg1', 'eeg2', 'eeg3', 'eeg4', 'eeg5', 'eeg6', 'eeg7', 'eeg8']
info = mne.create_info( ch_names=ch_names, sfreq=sampling_freq,ch_types=ch_types)
print(info)

ch_names = ['eeg1', 'eeg2', 'eeg3', 'eeg4', 'eeg5', 'eeg6', 'eeg7', 'eeg8']


<Info | 7 non-empty values
 bads: []
 ch_names: eeg1, eeg2, eeg3, eeg4, eeg5, eeg6, eeg7, eeg8
 chs: 8 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 125.0 Hz
 meas_date: unspecified
 nchan: 8
 projs: []
 sfreq: 250.0 Hz
>


In [6]:
raw_data = raw.get_data()
mne.filter.filter_data(raw_data,sfreq=250,l_freq=1,h_freq=40)

Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 825 samples (3.300 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished


array([[ 1.17324817e-10, -1.31377409e+02, -1.85107474e+02, ...,
        -3.98472615e+02, -2.79467944e+02,  1.37561074e-10],
       [ 1.73713488e-10, -1.32479137e+02, -1.87773445e+02, ...,
        -5.19937964e+02, -3.65202661e+02,  1.51203494e-10],
       [ 1.25510269e-10, -9.24035703e+01, -1.27679101e+02, ...,
        -2.35399122e+01, -1.78451081e+01,  1.55068847e-10],
       ...,
       [ 1.10048859e-10, -1.93404859e+02, -2.74331895e+02, ...,
        -8.39933256e+02, -5.96480433e+02,  1.70132353e-10],
       [ 1.08229870e-10, -1.20461140e+02, -1.70536797e+02, ...,
        -4.28990850e+02, -3.03902679e+02,  9.58948476e-11],
       [ 1.70075509e-10, -1.14881641e+02, -1.59792325e+02, ...,
        -4.42155873e+02, -3.14720576e+02,  1.47281298e-10]])

In [11]:
features = ['Coeffiecient of Variation','Mean of Vertex to Vertex Slope','Variance of Vertex to Vertex Slope',
         'Hjorth_Activity','Hjorth_Mobility','Hjorth_Complexity',
         'Kurtosis','2nd Difference Mean','2nd Difference Max',
         'Skewness','1st Difference Mean','1st Difference Max',
         'FFT Delta MaxPower','FFT Theta MaxPower','FFT Alpha MaxPower','FFT Beta MaxPower','Delta/Theta','Delta/Alpha','Theta/Alpha','(Delta+Theta)/Alpha',
         '1Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Approximate Energy','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Detailed Energy','Wavelet Approximate Entropy','Wavelet Detailed Entropy',
         '2Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Approximate Energy','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Detailed Energy','Wavelet Approximate Entropy','Wavelet Detailed Entropy',
         '3Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Approximate Energy','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Detailed Energy','Wavelet Approximate Entropy','Wavelet Detailed Entropy',
         '4Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Approximate Energy','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Detailed Energy','Wavelet Approximate Entropy','Wavelet Detailed Entropy',
         '5Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Approximate Energy','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Detailed Energy','Wavelet Approximate Entropy','Wavelet Detailed Entropy',
         '6Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Approximate Energy','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Detailed Energy','Wavelet Approximate Entropy','Wavelet Detailed Entropy',
         '7Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Approximate Energy','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Detailed Energy','Wavelet Approximate Entropy','Wavelet Detailed Entropy',
         '8Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Approximate Energy','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Detailed Energy','Wavelet Approximate Entropy','Wavelet Detailed Entropy',
         'AR1','AR2','AR3','AR4','AR5','AR6','AR7','AR8','AR9','AR10','AR11','AR12','AR13','AR14','AR15','AR16','AR17','AR18',
         'AR19','AR20','AR21','AR22','AR23','AR24']

In [13]:
files = []

user_data_path = working_dir/'User_data'
for user in os.listdir(user_data_path):
    for session in os.listdir(user_data_path/user):
        for file in os.listdir(user_data_path/user/session):
            if file.endswith('.csv') and not file.endswith('_HEP.csv'):
                files.append(user_data_path/user/session/file)

In [14]:
print(len(files))

20


In [16]:
features_csv_file = 'features.csv'
features_csv_file_path = data_dir/features_csv_file
#delete the file if it exists
if os.path.exists(features_csv_file_path):
    os.remove(features_csv_file_path)


with open(features_csv_file_path, "a",newline='') as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerow(features)
    for counter in range(len(files)):
        data_path = files[counter]
        print(data_path)
        # read the csv file and get a mne raw object
        data,label = csv_to_mne_raw(data_path)
        # get the data in ndarray
        raw_data = data.get_data()

        #apply a band pass filter of 1-40Hz
        filtered_data = mne.filter.filter_data(raw_data,sfreq=250,l_freq=1,h_freq=40)

        # sigbufs = [l for l in filtered_data]
        sigbufs = [l for l in raw_data]
        sigbufs = np.array(sigbufs)
        sigbufs = sigbufs.transpose()
        sigbufs = sigbufs.astype(float)

        print(sigbufs.shape)
        # continue

        #get the features
        for i in np.arange(1,150,3):
            temp_features = []
            epoch = sigbufs[i*250:(i+3)*250,:]
            if (len(epoch)==0 or len(epoch[0]) == 0):
                break
        
            #Coeffeicient of Variation
            temp_features.append(gf.coeff_var(epoch))

            #Mean of Vertex to Vertex Slope
            temp_features.append(gf.slope_mean(epoch))
            
            #Variance of Vertex to Vertex Slope
            temp_features.append(gf.slope_var(epoch))

            #Hjorth Parameters
            feature_list = gf.hjorth(epoch)
            for feat in feature_list:
                temp_features.append(feat)
            
            #Kurtosis
            temp_features.append(gf.kurtosis(epoch))

            #Second Difference Mean
            temp_features.append(gf.secDiffMean(epoch))

            #Second Difference Max
            temp_features.append(gf.secDiffMax(epoch))
               
            #Skewness
            temp_features.append(gf.skewness(epoch))
 
            #First Difference Mean
            temp_features.append(gf.first_diff_mean(epoch))

            #First Difference Max
            temp_features.append(gf.first_diff_max(epoch))

            # print(type(epoch))
            epoch_t = epoch.transpose()
            #FFT Max Power - Delta, Theta, Alpha & Beta Band!
            # feature_list  =  maxPwelch(epoch,128)            
            feature_list  =  gf.maxPwelch(epoch_t,250)            
            for feat in feature_list:
                temp_features.append(feat)
            #FFT Frequency Ratios
            temp_features.append(feature_list[0]/feature_list[1])
            temp_features.append(feature_list[0]/feature_list[2])
            temp_features.append(feature_list[1]/feature_list[3])
            temp_features.append((feature_list[0] + feature_list[1])/feature_list[2])
            
            #Wavelet Fetures!      lineterminator
            feature_list = gf.wavelet_features(epoch,8)
            for feat in feature_list:
                temp_features.append(feat)
                    
            #Autoregressive model Coefficients
            feature_list = gf.autogressiveModelParametersBurg(epoch)
            for feat in feature_list:
                temp_features.append(feat.real)

            temp_features.append(label)
            writer.writerow(temp_features)
        

/mnt/c/Users/Abhishek/Desktop/OpenBCI_DataAnlaysis/User_data/12345/session_1/12345_session_1_22-03-2023_21-53.csv
Creating RawArray with float64 data, n_channels=8, n_times=46250
    Range : 0 ... 46249 =      0.000 ...   184.996 secs
Ready.
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 825 samples (3.300 sec)

(46250, 8)
/mnt/c/Users/Abhishek/Desktop/OpenBCI_DataAnlaysis/User_data/12345/session_2/12345_session_2_22-03-2023_21-59.csv


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished


Creating RawArray with float64 data, n_channels=8, n_times=46250
    Range : 0 ... 46249 =      0.000 ...   184.996 secs
Ready.
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 825 samples (3.300 sec)

(46250, 8)
/mnt/c/Users/Abhishek/Desktop/OpenBCI_DataAnlaysis/User_data/12346/session_1/12346_session_1_22-03-2023_22-21.csv


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished


Creating RawArray with float64 data, n_channels=8, n_times=46250
    Range : 0 ... 46249 =      0.000 ...   184.996 secs
Ready.
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 825 samples (3.300 sec)

(46250, 8)
/mnt/c/Users/Abhishek/Desktop/OpenBCI_DataAnlaysis/User_data/12346/session_2/12346_session_2_22-03-2023_22-26.csv


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished


Creating RawArray with float64 data, n_channels=8, n_times=46250
    Range : 0 ... 46249 =      0.000 ...   184.996 secs
Ready.
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 825 samples (3.300 sec)

(46250, 8)
/mnt/c/Users/Abhishek/Desktop/OpenBCI_DataAnlaysis/User_data/12355/session_2/12355_session_2_03-04-2023_23-43.csv


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished


Creating RawArray with float64 data, n_channels=8, n_times=46250
    Range : 0 ... 46249 =      0.000 ...   184.996 secs
Ready.
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 825 samples (3.300 sec)

(46250, 8)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished


NORMALISATION

In [12]:
r = csv.reader(open('features.csv')) 
lines = [l for l in r]

In [13]:
for i in range(len(lines[1])-1):  
    columns = []
    for j in range(1,len(lines)):
        columns.append(float(lines[j][i]))
    mean = np.mean(columns,axis = 0)
    #print('\nMean = ',mean)
    std_dev  = np.std(columns,axis = 0)
    #print('\nSTD Deviation = ',std_dev)
    for j in range(1,len(lines)):
        lines[j][i] = (float(lines[j][i])-mean)/std_dev

writer = csv.writer(open('Normalizedfeatures.csv', 'w'))
writer.writerows(lines)
print('Done!')

Done!
